# Word2Vec ile duygu analizi

### Alıştırma hedefleri:
- Word2Vec ile kelimeleri vektörlere dönüştürmek
- Word2vec tarafından verilen kelime temsilini RNN'ye beslemek için kullanmak

<hr>

▶️ Bu hücreyi çalıştırın ve kullandığınız 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) sürümünün ≥ 4.0 olduğundan emin olun!

In [ ]:
!pip freeze | grep gensim

# Veri


❓ **Soru** ❓ Öncelikle verileri yükleyelim. Fonksiyonda neler olduğunu anlamanıza gerek yok, burada önemi yok.

⚠️ **Uyarı** ⚠️ `load_data` fonksiyonunda `percentage_of_sentences` argümanı vardır. Bilgisayarınıza bağlı olarak, çok fazla cümle bilgisayarınızı yavaşlatabilir veya hatta dondurabilir - RAM'iniz taşabilir. Bu nedenle, **cümlelerin %10'uyla başlamalı** ve bilgisayarınızın bunu kaldırabildiğini kontrol etmelisiniz. Aksi takdirde, daha düşük bir sayı ile yeniden çalıştırın. 

⚠️ **DISCLAIMER** ⚠️ **_En büyüğü kimde_ (_who has the biggest_)(RAM) oyununu oynamaya gerek yok!** Buradaki amaç, modellerinizi hızlı bir şekilde çalıştırarak prototip oluşturmaktır. Gerçek hayatta bile, hızlı bir şekilde döngü ve hata ayıklama yapmak için verilerinizin bir alt kümesiyle başlamanız önerilir. Bu nedenle, yalnızca en iyi doğruluğu elde etmek istiyorsanız sayıyı artırın.

In [ ]:
import numpy as np

In [ ]:
####################################################
### Verileri yüklemek için bu hücreyi çalıştırın ###
####################################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Tüm verilerin yalnızca belirli bir yüzdesini alın
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

Önceki alıştırmada, Word2vec temsilini eğittiniz ve bu Şekil'in ilk adımında gösterildiği gibi, tüm eğitim cümlelerinizi bir RNN'ye beslemek için dönüştürdünüz: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/word2vec_representation.png" alt="Word2Vec with RNN" width="400px" />



❓ **Soru** ❓ Burada, önceki alıştırmada yaptığınızın aynısını tekrar yapalım. İlk olarak, eğitim cümleniz üzerinde bir word2vec modeli (istediğiniz argümanlarla) eğitin. Bunu `word2vec` değişkenine kaydedin.

In [ ]:
# SENİN KODUN BURAYA

Önceki alıştırmadaki işlevleri yeniden kullanarak, eğitim ve test verilerinizi RNN'ye girebileceğiniz bir biçime dönüştürelim.

❓ **Soru** ❓ Neler olduğunu anladığınızdan emin olmak için aşağıdaki işlevi okuyun ve çalıştırın.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Bir cümleyi (kelime listesi) gömme uzayındaki kelimeleri temsil eden bir matrise dönüştürme işlevi
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return np.array(embedded_sentence)

# Bir cümle listesini matris listesine dönüştüren işlev
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Eğitim ve test cümlelerini gömün(embed edin)
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Eğitim ve test gömülü cümleleri doldurun
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

☝️ Çalıştığından emin olmak için, `X_train_pad` ve `X_test_pad` için aşağıdakileri kontrol edelim:

- bunlar numpy dizileridir
- bunlar 3 boyutludur
- son boyut, word2vec gömme alanınızın boyutundadır (bunu `word2vec.wv.vector_size` ile elde edebilirsiniz
- ilk boyut, `X_train` ve `X_test` boyutundadır

✅ **İyi Uygulama** ✅ Bu tür testler oldukça önemlidir! Sadece bu alıştırmada değil, gerçek hayattaki uygulamalarda da. Hataları çok geç fark etmeyi ve bunların tüm not defterine yayılmasını önler.

In [ ]:
# BENİ TEST ET
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Temel model

Kendi modelinizi test etmek için çok basit bir modele sahip olmak her zaman iyidir - çok basit bir algoritmadan daha iyi bir şey yaptığınızdan emin olmak için.

❓ **Soru** ❓ Temel doğruluk oranınız nedir? Bu durumda, temeliniz `y_train` içinde en çok bulunan etiketi tahmin etmek olabilir (tabii ki, veri kümesi dengeli ise, temel doğruluk oranı 1/n'dir; burada n, sınıfların sayısıdır - burada 2'dir).

In [ ]:
# SENİN KODUN BURAYA

# Model

❓ **Soru** ❓ Aşağıdaki katmanlara sahip bir RNN yazın:
- bir `Masking` katmanı
- 20 birim ve `tanh` aktivasyon fonksiyonuna sahip bir `LSTM`
- 10 birimlik bir `Dense`
- görevinize bağlı bir çıktı katmanı

Ardından, modelinizi derleyin (en azından başlangıçta optimizer olarak `rmsprop` kullanmanızı öneririz).

In [ ]:
# SENİN KODUN BURAYA

❓ **Soru** ❓ Modeli gömülü ve doldurulmuş verilerinize uyarlayın - erken durdurma kriterini unutmayın.

❗ **Not** ❗ Doğruluğunuz büyük ölçüde eğitim kümenize bağlı olacaktır. Burada, performansınızın temel modelin üzerinde olduğundan emin olun (bu, başlangıçtaki IMDB verilerinin yalnızca %20'sini yüklemiş olsanız bile geçerli olmalıdır).

In [ ]:
# SENİN KODUN BURAYA

❓ **Soru** ❓ Test setinde modelinizi değerlendirin

In [ ]:
# SENİN KODUN BURAYA

# Eğitimli Word2Vec - Transfer Öğrenimi

Doğruluğunuz, temel modelin üzerinde olsa da, oldukça düşük olabilir. Veri temizleme ve gömme kalitesini iyileştirme gibi bunu iyileştirmek için birçok seçenek vardır.

Burada veri temizleme stratejilerine girmeyeceğiz. Gömme kalitemizi iyileştirmeye çalışalım. Ancak, daha büyük bir metin kümesini yüklemek yerine, neden başkalarının öğrendiği gömme modelinden yararlanmayalım? Çünkü gömme modelinin kalitesi, yani kelimelerin yakınlığı, farklı görevlerden elde edilebilir. Transfer öğrenme tam olarak budur.

❓ **Soru** ❓ Bu sayede word2vec'te bulunan tüm farklı modelleri listeleyin: 

In [ ]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

ℹ️ Modellerin listesini ve boyutlarını [`gensim-data` deposunda](https://github.com/RaRe-Technologies/gensim-data#models) de bulabilirsiniz.

❓ **Soru** ❓ Önceden eğitilmiş word2vec gömme alanlarından birini yükleyin.

Bunu `api.load(seçtiğiniz model)` ile yapabilir ve `word2vec_transfer` içinde saklayabilirsiniz.

<details>
    <summary>💡 İpucu</summary>
    
`glove-wiki-gigaword-50` modeli, daha küçük olması (65 MB) nedeniyle başlangıç için iyi bir seçenektir.

</details>

In [ ]:
# SENİN KODUN BURAYA

❓ **Soru** ❓ Kelime dağarcığının boyutunu ve aynı zamanda gömme alanının boyutunu kontrol edin.

In [ ]:
# SENİN KODUN BURAYA

❓ İlk soruda yaptığımız gibi, `X_train` ve `X_test`'i gömelim! (`embed_sentence_with_TF` işlevinde küçük bir fark var, ancak bu konuya girmeyeceğiz.)

In [ ]:
# Bir cümleyi (kelime listesi) gömme uzayındaki kelimeleri temsil eden bir matrise dönüştürme işlevi
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])

    return np.array(embedded_sentence)

# Bir cümle listesini matris listesine dönüştüren işlev
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Eğitim ve test cümlelerini gömün (embed edin)
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Soru** ❓  Sonuçlarınızı doldurmayı ve bunları `X_train_pad_2` ve `X_test_pad_2` içinde saklamayı unutmayın.

In [ ]:
# SENİN KODUN BURAYA

❓ **Soru** ❓ Modeli yeniden başlatın ve yeni gömülü (ve doldurulmuş(padded)) verilerinize uyarlayın!  Test setinizde değerlendirin ve önceki doğruluğunuzla karşılaştırın.

❗ **Not** ❗ Buradaki eğitim biraz zaman alabilir. Sadece 10 dönem hesaplayabilir (bu **iyi** bir uygulama değildir, sadece çok uzun süre beklememek içindir) ve eğitim sürerken bir sonraki alıştırmaya geçebilir veya bir mola verebilirsiniz, muhtemelen bunu hak ettiniz ;)

In [ ]:
# SENİN KODUN BURAYA

In [ ]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Yeni word2vec'iniz büyük bir metin kümesinde eğitildiğinden, çok sayıda kelimeyi temsil eder! Küçük veri kümenize kıyasla çok daha fazladır, özellikle de eğitim kümesinde belirli bir sayıdan fazla bulunmayan kelimeleri elediğiniz için. Bu nedenle, eğitim ve test kümenizde çok daha fazla gömülü kelime vardır, bu da her yinelemeyi öncekinden daha uzun hale getirir.